In [ ]:
from dotenv import get_key
from pymongo import MongoClient
import pymongo
import psycopg2 # Remover
from pprint import pprint  # Imprimir bsons formatados
from utils import *

STRING = get_key('.env', 'MG_STRING')
NAME = get_key('.env', 'PG_NAME')
USER = get_key('.env', 'PG_USER')
PASSWORD = get_key('.env', 'PG_PASSWORD')
HOST = get_key('.env', 'PG_HOST')
PORT = get_key('.env', 'PG_PORT')

## Funções para acesso ao MongoDB

In [ ]:
def create_mongo_connection():
	"""Cria uma conexão com o banco de dados MongoDB. Retorna o cliente.
	AVISO: Se a conexão não funcionar, verifique no site do atlas se seu IP atual está autorizado a se conectar."""
	try:
		client = MongoClient(STRING, server_api=pymongo.server_api.ServerApi(version="1"))
		client.admin.command("ping")
		print("---- Conexão bem sucedida ----")
	except Exception as e:
		raise Exception(
			"The following error occurred: ", e)
	return client

def end_mongo_connection(client):
	client.close()
	print("---- Conexão encerrada -----")

## Criação de tabelas no banco

In [ ]:
def create_tables(db, col):
	match col:
		case 'pais':
			db.command({
				"collMod": "pais",
				"validator": {
					"$jsonSchema": {
						"bsonType": "object",
						"properties": {
							"nome": {"bsonType": "string"},
							"codigo": {"bsonType": "string"},
							"continente": {"bsonType": "string"},
							"grupos": {
								"bsonType": "array",
								"items": {
									"bsonType": "object",
									"properties": {
										"sigla": {"bsonType": "string"}
									}
								}
							},
							"_id": {"bsonType": "objectId"}
						},
						"title": "pais"
					}
				},
				'validationLevel': 'moderate',
				'validationAction': 'warn'
			})
		case 'emissaoGas':
			collection_name = 'emissaoGas'
		case _:
			raise ValueError(f"Nome de coleção inválido: {col}")

In [ ]:
def consultas(client):
	# for db_info in client.list_database_names():
	# 	print(db_info)
	# db = client["Biblioteca"]
	# # collections = db.list_collection_names()
	# # print("Coleções em samples_mflix:", collections)
	# result = db.books.find({"authors": "Machado de Assis"}, {'_id': 0, 'title': 1})
	
	# for doc in result:
	# 	pprint(doc)

	db = client["projeto_mc536"]
	# colecao = db["teste"]
	# # db.teste.insertOne({'name': 'José',
	# # 				 'age': 58})
	# colecao.insert_one({'campo': 'valor exemplo'})
	# result = db['teste'].find({}, {'_id':0})
	# for doc in result:
	# 	pprint(doc)


	# result = db['teste'].find({}, {'_id':0})
	# print('---')
	# for doc in result:
	# 	pprint(doc)

	# Corrigindo a criação da coleção 'pais' caso ela não exista antes de aplicar o validator

	query = 'SELECT * FROM PAIS;'
	conn = create_pg_connection()
	cursor = conn.cursor()
	cursor.execute(query)
	rows = cursor.fetchall()
	for row in rows:
		print(row)

	'''pais {
	sigla:
	nome:
	continente:
	grupos: String[]
	estatisticas: {
		ano:
		pib:
		geografia: {
			area_tipo:
			num_habitantes:
			area_km2:
		} []
	} []
}'''

	entries = {'pais': [
			{
				'continente': 18,
				'codigo': 'HW',
				'nome': 'Hogwarts',
				'grupos': [{'sigla': 'MAGIA'}],
				'preco' : 15
			}
		],
		'emissaoGas': [
			{
				'paisCodigo': 'BRA',
				'ano': 2005,
				'emissoes': [{
					'nome': 'Dióxido de Carbono',
					'formula': 'CO2',
					'valor': 320
				}]
			},
			{
				'paisCodigo': 'USA',
				'ano': 1998,
				'emissoes': [{
					'nome': 'Metano',
					'formula': 'CH4',
					'valor': 1000
				}]
			}
		]}
	
	for col in entries.keys():

		if col not in db.list_collection_names():
			db.create_collection(col)

		create_tables(db, col)
		db[col].delete_many({})
		db[col].insert_many(entries[col])

		print(f'> Coleção {col} <')
		# Exibindo os documentos inseridos
		for doc in db[col].find({}, {'_id': 0}):
			pprint(doc)

client = create_mongo_connection()
consultas(client)
end_mongo_connection(client)


In [ ]:
client = create_mongo_connection()
consultas(client)
end_mongo_connection(client)

In [ ]:
query = """
SELECT * FROM PAIS;
"""

conn = create_pg_connection()
cursor = conn.cursor()
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
	print(row[1])
end_pg_connection(conn)

In [ ]:
conn = create_pg_connection()
cursor = conn.cursor()

query = 'SELECT * FROM PAIS;'
cursor.execute(query)
rows = cursor.fetchall()
paises = []
for row in rows:
	paises.append({
					'sigla': row[0],
					'nome': row[1],
					'continente': row[2],
					'estatisticas': []
				})
# Grupos
for pais in paises:
	# if pais['sigla'] != 'AFG':
	# 	continue
	query = f'''SELECT SIGLA
	FROM
	GRUPO G INNER JOIN PAIS_GRUPO PG
		ON G.ID_GRUPO = PG.GRUPO_ID_GRUPO
	WHERE PAIS_CODIGO = '{pais['sigla']}';'''
	cursor.execute(query)
	rows = cursor.fetchall()
	grupos = [row[0] for row in rows]
	if len(grupos) > 0:
		pais['grupos'] = grupos

	query = f'''SELECT ANO,
	VALOR_USD
	FROM PIB
	WHERE CODIGO_PAIS = '{pais['sigla']}'
	ORDER BY ANO;'''
	cursor.execute(query)
	rows = cursor.fetchall()

	for row in rows:
		ano, pib = row
		# print(ano, pib, pais['sigla'])
		geografias = []
		# Áreas e populações urbana, rural e total
		subquery = f'''SELECT TIPO,
		NUM_HABITANTES,
		AREA_KM2
		FROM
		AREA A INNER JOIN DEMOGRAFIA D
			ON D.AREA_ID = A.ID
		INNER JOIN DISTRIBUICAO_TERRITORIAL DT
			ON DT.AREA_ID = A.ID AND DT.PAIS_CODIGO = D.PAIS_CODIGO AND DT.ANO = D.ANO
		WHERE DT.PAIS_CODIGO = '{pais['sigla']}'
			AND DT.ANO = {ano}
		ORDER BY DT.ANO, TIPO;'''
		cursor.execute(subquery)
		subrows = cursor.fetchall()
		for sr in subrows:
			geografia = {
				'area_tipo': sr[0],
				'num_habitantes': sr[1],
				'area_km2': sr[2]
			}
			geografias.append(geografia)
		
		# Área florestal
		subquery = f'''SELECT TIPO,
		AREA_KM2
		FROM
		AREA A INNER JOIN DISTRIBUICAO_TERRITORIAL DT
			ON DT.AREA_ID = A.ID
		WHERE DT.PAIS_CODIGO = '{pais['sigla']}'
			AND DT.ANO = {ano}
		AND TIPO = 'Forest'
		ORDER BY DT.ANO, TIPO;'''
		cursor.execute(subquery)
		subrow = cursor.fetchall()
		for sr in subrow:
			geografia = {
				'area_tipo': sr[0],
				'area_km2': sr[1]
			}
			geografias.append(geografia)

		est = {
			'ano': row[0],
			'pib': row[1]
		}
		if geografias:
			est['geografias'] = geografias
		pais['estatisticas'].append(est)

end_pg_connection(conn)

pprint(paises)


In [ ]:
conn = create_pg_connection()
cursor = conn.cursor()

emissoes = []

query = 'SELECT * FROM GAS;'
cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
	id, nome, formula = row
	emissao = {
		'nome': nome,
		'formula': formula,
		'medicoes': []
	}

	subquery = f'''SELECT PAIS_CODIGO,
	ANO,
	EMISSAO_MTCO2
	FROM EMISSAO_GAS
	WHERE GAS_ID_GAS = {id};'''
	cursor.execute(subquery)
	subrows = cursor.fetchall()

	for sr in subrows:
		cod_pais, ano, valor = sr
		emissao['medicoes'].append({
			'cod_pais': cod_pais,
			'ano': ano,
			'valor_mtco2': valor
		})
	
	emissoes.append(emissao)

pprint(emissoes)


end_pg_connection(conn)

In [ ]:
conn = create_pg_connection()
cursor = conn.cursor()

energias = []

query = 'SELECT * FROM INTERACAO_ENERGETICA;'
cursor.execute(query)
rows = cursor.fetchall()

for row in rows:
	id_interacao, interacao, unidade = row
	energia = {
		'interacao': interacao,
		'unidade': unidade,
		'fontes': []
	}
	
	query = 'SELECT * FROM ENERGIA;'
	cursor.execute(query)
	subrows = cursor.fetchall()

	for sr in subrows:
		id_energia, nome = sr
		fonte = {
			'nome': nome,
			'medicoes': []
		}
		query = f'''SELECT CODIGO_PAIS,
		ANO,
		VALOR
		FROM RELATORIO_ENERGETICO
		WHERE ID_INTERACAO = {id_interacao}
			AND ID_ENERGIA = {id_energia};'''
		cursor.execute(query)
		registros = cursor.fetchall()

		for reg in registros:
			cod_pais, ano, valor = reg
			fonte['medicoes'].append({
				'cod_pais': cod_pais,
				'ano': ano,
				'valor': valor
			})

		energia['fontes'].append(fonte)

	energias.append(energia)

pprint(energias)

end_pg_connection(conn)